<a href="https://colab.research.google.com/github/preethi2076/Python_list/blob/main/AD3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
#create  text data with emoji
text1 = "What are you saying 😂.Iam the boss 😎,and why are you so 😒"

In [ ]:
import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

In [ ]:
# Function for converting emojis into word
def converting_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","")\
                                           .replace(":","").split()))
    return text

converting_emojis(text1)

'What are you saying face_with_tears_of_joy.Iam the boss smiling_face_with_sunglasses,and why\xa0are\xa0you\xa0so\xa0unamused_face'

In [ ]:
def emoji_removal(string):
  emoji_unicode = re.compile("["
                            u"\U0001F600-\U0001F64F"
                            u"\U0001F300-\U0001F5FF"
                            u"\U0001F680-\U0001F6FF"
                            u"\U0001F1E0-\U0001F1FF"
                            u"\U00002500-\U00002BEF"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001F926-\U0001F937"
                            u"\U00010000-\U0010FFFF"
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"
                            u"\u3030"
                            "]+", flags=re.UNICODE)
  return emoji_unicode.sub(r'', string)

emoji_removal(text1)

'What are you saying .Iam the boss ,and why\xa0are\xa0you\xa0so\xa0'

In [ ]:
import pandas as pd
dataset=pd.read_csv('/content/hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
dataset.shape

(5242, 3)

In [ ]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [ ]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [ ]:
import re
def clean_text(text):
  text=re.sub(r'[^a-zA-Z\']',' ',text)
  text=re.sub(r'[^\x00-\x7F]+','',text)
  text=text.lower()
  return text


In [ ]:
dataset['clean_text']=dataset.tweet.apply(lambda x:clean_text(x))

In [ ]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i'm it's so gr...


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [ ]:
stop=stopwords.words('english')

In [ ]:
def gen_freq(text):
  word_list=[]
  for tw_words in text.split():
    word_list.extend(tw_words)
  word_freq=pd.Series(word_list).value_counts()
  word_freq=word_freq.drop(stop,errors='ignore')
  return word_freq

In [ ]:
def any_neg(words):
  for word in words:
    if word in ['n','no','non','not'] or re.search(r"\wn't",word):
      return 1
    else:
      return 0

In [ ]:
def any_rare(words,rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [ ]:
def is_question(words):
  for word in words:
    if word in ['when','what','how','why','who']:
      return 1
    else:
      return 0

In [ ]:
word_freq=gen_freq(dataset.clean_text.str)
rare_100=word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x:len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x:any_neg(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x:any_rare(x,rare_100))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x:is_question(x))
dataset['char_count']=dataset.clean_text.apply(lambda x:len(x))

In [ ]:
wl={'text':['hi','how','are','you','I','am','good','it','how','are','you']}
df=pd.DataFrame(wl)
df

,text
0,hi
1,how
2,are
3,you
4,I
5,am
6,good
7,it
8,how
9,are
